# **General Matrix Multiplication (GEMM) Optimization**

In [ ]:
import os
os.environ["PATH"] += ":/usr/local/cuda/bin"
!nvcc --version

In [ ]:
import re
import subprocess
import statistics

def run_gemm(executable_path, choice, m, n, k, runs=10):
   times = []
   for i in range(runs):
       result = subprocess.run([executable_path, str(choice), str(m), str(n), str(k)],
                             capture_output=True, text=True)
       #print(f"Debug output: {result.stdout}")
       match = re.search(r'CUDA kernel time: (\d+\.\d+)', result.stdout)
       if match:
           cuda_time = float(match.group(1))
           print(f"Run-{i+1}: {cuda_time} ms")
           times.append(cuda_time)
       else:
           print(f"Warning: No time found in output: {result.stdout}")
           
   return statistics.mean(times)


In [ ]:
!nvcc -o ./src/gemm ./src/run.cu -lcublas

## **1 - Naive GEMM Kernel**

<div style="text-align: center;">
  <img src="./images/naive_kernel_mul.png" alt="Naive GEMM Multiplication" width="800">
</div>

This diagram shows a naive GEMM (General Matrix Multiplication) kernel implementation using threads. Each thread accesses matrix elements based on its ID: x = blockDim.x * blockIdx.x + threadIdx.x and y = blockDim.y * blockIdx.y + threadIdx.y. Within the B matrix, threads in a warp access the same values (broadcast), while in the A matrix, threads access non-consecutive memory locations (non-coalesced memory access), which is inefficient. The C matrix shows how different threads (0,0), (0,1), (0,2) etc., compute their respective output elements through these memory access patterns.

<div style="text-align: center;">
  <img src="./images/naive_kernel_memory_access.png" alt="Naive Kernel Memory Access" width="800">
</div>

This diagram illustrates a memory access pattern issue in GPU computing. It shows two warps (groups of threads) accessing memory in a non-coalesced pattern, meaning threads access scattered memory locations rather than consecutive ones. Each warp requires 4x32B loads (8 loads total), which is inefficient. The crossing lines between thread indices and memory locations visualize this scattered access pattern. This non-optimal memory access results in performance penalties because too many separate load operations are needed to execute each warp.

In [ ]:
!nvcc -o ./src/01_naive_gemm ./src/run.cu -lcublas

In [ ]:
naive_gemm_time = run_gemm("./src/01_naive_gemm", 1, 4096, 4096, 4096)
print(f"Average Naive GEMM time: {naive_gemm_time}")

## **2 - Coalesced Memory GEMM Kernel**

<div style="text-align: center;">
  <img src="./images/coalesced_memory_mul.png" alt="Coalesced Memory Multiplication" width="800">
</div>

This diagram shows an optimized memory coalesced GEMM (General Matrix Multiplication) kernel design. Unlike the naive version, threads access consecutive memory locations in matrix B, enabling memory coalescing and better performance. For matrix A, all threads within a warp access the same values (broadcast). The coordinates are calculated as: x = blockIdx.x * BLOCK_SIZE + (threadIdx.x / BLOCK_SIZE) for matrix A's row access, and y = blockIdx.y * BLOCK_SIZE + (threadIdx.y % BLOCK_SIZE) for matrix B's column access. Threads (0,0), (0,1), and (0,2) are grouped in the same warp to optimize memory access patterns.

<div style="text-align: center;">
  <img src="./images/coalesced_memory_access.png" alt="Coalesced Memory Access" width="800">
</div>

This diagram shows an optimized memory coalesced access pattern where threads within each warp (Warp-0 and Warp-1) access consecutive memory locations. Each warp now requires only 2x32B loads (4 loads total), half of what was needed in the non-coalesced version. The straight vertical lines from thread indices to memory locations indicate efficient coalesced memory access, improving performance by reducing the number of required load operations.

In [ ]:
!nvcc -o ./src/02_memory_coalesced_gemm ./src/run.cu -lcublas

In [ ]:
memory_coalesced_gemm_time = run_gemm("./src/02_memory_coalesced_gemm", 2, 4096, 4096, 4096)
print(f"Average Naive GEMM time: {memory_coalesced_gemm_time}")

## **3 -  Shared Memory Cache-Blocking**

<div style="text-align: center;">
  <img src="./images/shared_memory_cache_blocking.png" alt="Shared Memory Cache Blocking" width="800">
</div>

This diagram illustrates a block-based matrix multiplication algorithm with a block size of 32. When multiplying matrices A and C, each matrix is divided into blocks of size BLOCK_SIZE (32). The starting addresses of blocks are calculated using formulas: &A = row * BLOCK_SIZE * K for matrix A's rows, &B = col * BLOCK_SIZE for B's columns, and &C = (row * BLOCK_SIZE * K) + (col * BLOCK_SIZE) for matrix C's position. As the algorithm processes each block, it increments A by BLOCK_SIZE within the same row, B by BLOCK_SIZE * N to move to the next block, and C moves to process the next row of blocks.

In [ ]:
!nvcc -o ./src/03_shared_memory_gemm ./src/run.cu -lcublas

In [ ]:
shared_memory_gemm_time = run_gemm("./src/03_shared_memory_gemm", 3, 4096, 4096, 4096)
print(f"Average Naive GEMM time: {shared_memory_gemm_time}")